# Satellite visibility map
### Guillaume Witz, Science IT Support, Bern University

In [1]:
#import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, timedelta

from ipywidgets import interact
import ipywidgets as ipw

import aiub

In [13]:
#user paramters

#file location
address_sat = 'http://ftp.aiub.unibe.ch/CODE_MGEX/CODE/'
#earth radius
mean_rad = 6369915.831429518

In [4]:
#Create a grid of points and calculate their local ellipsoidal coordinates.
coord_table, lons, lats = aiub.coord_grid(nlats=20, nlons=20)

In [5]:
#Pick dates
date_min, date_max = aiub.date_picker()

In [6]:
#create a list of dates between start and end date 
date_list = [date_min.value + timedelta(days=x) for x in range((date_max.value-date_min.value).days+1)]
#calculate gps weeks times (year, week, day)
gps_weeks = [aiub.date_to_gpsweeks(x) for x in date_list]

#load satellite data
temp_pd = pd.concat([aiub.import_RM_file(address_sat, g) for g in gps_weeks]).reset_index()

sat_dict = {'G':'GPS','R':'GLONASS','E':'Galileo','C':'BeiDou','J':'QZSS'}
temp_pd['satellite'] = temp_pd.satellite.apply(lambda x : sat_dict[x])
all_dates = temp_pd.datetime.unique()

In [7]:
#Calculate elevation and azimuth for all pairs of coordinates and satellites.
el_az = aiub.elevation_azimuth_grid(coord_table, temp_pd)

In [8]:
#Group observations by time
grouped = el_az[(el_az.e>np.deg2rad(5))].groupby('datetime')

In [9]:
%matplotlib inline

In [11]:
interact(aiub.plot_map,t = ipw.IntSlider(value = 0, min = 0, max = len(el_az.datetime.unique()),step =1, continuous_update = False), 
         sat = list(temp_pd.satellite.unique()),temp_pd = ipw.fixed(temp_pd), 
         all_dates = ipw.fixed(all_dates), coord_table = ipw.fixed(coord_table),
        lons = ipw.fixed(lons), lats = ipw.fixed(lats));


interactive(children=(IntSlider(value=0, continuous_update=False, description='t', max=867), Dropdown(descript…